In [1]:
import psycopg2
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Connect to Database

In [3]:
#Connect to PgAdmin
engine = psycopg2.connect(
    database="project-data",
    user = "postgres",
    password = "Post23!Fnord",
    host = "happinessproject.cqkpnj5dcmou.us-east-1.rds.amazonaws.com",
    port = '5432'
    )

cursor = engine.cursor()


In [4]:
#Pull combined data table
query = "SELECT * from combined_data"

df = pd.read_sql(query, con=engine)

In [5]:
df.head()

,country_name,region,ladder_score,stand_err,up_whisker,low_whisker,log_gdp_per_cap,social_support,healty_life_exp,freedom,...,ladder_2010,ladder_2012,ladder_2013,ladder_2014,ladder_2015,ladder_2016,ladder_2017,ladder_2018,ladder_2019,ladder_2020
0,Brazil,Latin America and Caribbean,6.330,0.043,6.415,6.245,9.577,0.882,66.601,0.804,...,6.837,7.038,7.140,6.981,6.547,6.375,6.333,6.191,6.451,6.110
1,India,South Asia,3.819,0.026,3.869,3.769,8.755,0.603,60.633,0.893,...,4.989,4.635,4.428,4.424,4.342,4.179,4.046,3.818,3.249,4.225
2,Russia,Commonwealth of Independent States,5.477,0.033,5.541,5.413,10.189,0.903,64.703,0.718,...,5.385,5.389,5.537,6.037,5.996,5.855,5.579,5.514,5.441,5.495
3,South Africa,Sub-Saharan Africa,4.956,0.060,5.074,4.839,9.403,0.860,56.904,0.749,...,4.652,4.931,3.661,4.828,4.887,4.770,4.514,4.884,5.035,4.947
4,Mexico,Latin America and Caribbean,6.317,0.053,6.420,6.213,9.859,0.831,68.597,0.862,...,6.802,6.910,7.443,6.680,6.236,6.824,6.410,6.550,6.432,5.964


# Clean master table for Machine Learning

In [7]:
#column names
list(df)

['country_name',
 'region',
 'ladder_score',
 'stand_err',
 'up_whisker',
 'low_whisker',
 'log_gdp_per_cap',
 'social_support',
 'healty_life_exp',
 'freedom',
 'generosity',
 'percept_corrupt',
 'ladder_score_distopia',
 'expby_log_gdp_per_cap',
 'expby_social_support',
 'expby_healty_life_exp',
 'expby_freedom',
 'expby_generosity',
 'expby_percept_corrupt',
 'distopia_plus_resid',
 'ac_both_sexes',
 'ac_male',
 'ac_female',
 'population_2021',
 'covid_total_cases',
 'covid_new_cases',
 'covid_total_deaths',
 'covid_new_deaths',
 'covid_total_recovered',
 'covid_new_recovered',
 'covid_active_cases',
 'covid_serious_critical',
 'covid_cases_per_mil',
 'covid_deaths_per_mil',
 'covid_total_tests',
 'covid_tests_per_mil',
 'covid_who_region',
 'population_2020',
 'land_area_skm',
 'density_skm',
 'meat_consumption',
 'median_age',
 'screen_time_avg',
 'suicide_rate',
 'ladder_2010',
 'ladder_2012',
 'ladder_2013',
 'ladder_2014',
 'ladder_2015',
 'ladder_2016',
 'ladder_2017',
 'ladde

In [8]:
df2 = df.filter(['ladder_score', 'freedom','healty_life_exp', 'log_gdp_per_cap', 'meat_consumption',
                 'percept_corrupt', 'social_support', 'covid_tests_per_mil', 'ac_female', 'generosity', 'covid_deaths_per_mil', 'covid_total_tests', 'covid_active_cases'], axis=1)

In [9]:
# view total null values for columns
df2.isna().sum()

ladder_score             0
freedom                  0
healty_life_exp          0
log_gdp_per_cap          0
meat_consumption        12
percept_corrupt          0
social_support           0
covid_tests_per_mil     23
ac_female                8
generosity               0
covid_deaths_per_mil    17
covid_total_tests       23
covid_active_cases      17
dtype: int64

In [10]:
#compare NA values to dataset row count
len(df2)

149

In [11]:
#drop all rows containing NA values
df2 = df2.dropna()

In [12]:
# view total null values for columns
df2.isna().sum()

ladder_score            0
freedom                 0
healty_life_exp         0
log_gdp_per_cap         0
meat_consumption        0
percept_corrupt         0
social_support          0
covid_tests_per_mil     0
ac_female               0
generosity              0
covid_deaths_per_mil    0
covid_total_tests       0
covid_active_cases      0
dtype: int64

In [13]:
#length of final dataset for analysis 
len(df2)

116

# Random Forest: Find the 10 variables that impact happiness scores the most 

In [14]:
# Create columns and target variables
columns = [
    'ladder_score', 'freedom','healty_life_exp', 'log_gdp_per_cap', 'meat_consumption',
                 'percept_corrupt', 'social_support', 'covid_tests_per_mil', 'ac_female', 
    'generosity', 'covid_deaths_per_mil', 'covid_total_tests', 'covid_active_cases'
]

target = ["ladder_score"]

In [15]:
#Convert variable type for target to non-continuous dtype
df2 = df2.astype({'ladder_score':'int'})

df2.head()

,ladder_score,freedom,healty_life_exp,log_gdp_per_cap,meat_consumption,percept_corrupt,social_support,covid_tests_per_mil,ac_female,generosity,covid_deaths_per_mil,covid_total_tests,covid_active_cases
0,6,0.804,66.601,9.577,82.4,0.756,0.882,62085.0,3.22,-0.071,464.0,13206188.0,771258.0
1,3,0.893,60.633,8.755,5.2,0.774,0.603,16035.0,1.89,0.089,30.0,22149351.0,606387.0
2,5,0.718,64.703,10.189,51.0,0.845,0.903,203623.0,4.31,-0.111,100.0,29716907.0,180931.0
3,4,0.749,56.904,9.403,39.0,0.860,0.860,53044.0,3.46,-0.067,162.0,3149807.0,141264.0
4,6,0.862,68.597,9.859,58.6,0.799,0.831,8189.0,2.18,-0.147,391.0,1056915.0,103325.0


In [16]:
# Create our features
X = pd.get_dummies(df2.drop(columns="ladder_score"))


# Create our target
y = df2["ladder_score"]

In [17]:
X.describe()

,freedom,healty_life_exp,log_gdp_per_cap,meat_consumption,percept_corrupt,social_support,covid_tests_per_mil,ac_female,generosity,covid_deaths_per_mil,covid_total_tests,covid_active_cases
count,116.00000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.00000,116.000000,116.000000,1.160000e+02,116.000000
mean,0.79681,65.128922,9.456112,45.112069,0.731862,0.816405,70759.991379,2.59150,-0.030078,92.468793,1.437235e+06,28377.758621
std,0.11199,6.527888,1.130718,34.220526,0.183552,0.117265,120702.292380,2.00247,0.150833,148.045397,3.750990e+06,94536.318327
min,0.38200,48.700000,6.635000,3.100000,0.082000,0.463000,4.000000,0.00000,-0.288000,0.080000,1.200000e+02,0.000000
25%,0.73100,60.427750,8.572750,17.125000,0.679750,0.759000,8405.750000,0.78750,-0.137500,6.000000,1.247078e+05,541.500000
50%,0.81050,66.651500,9.580500,34.950000,0.795500,0.834000,35939.000000,2.16000,-0.053500,29.500000,3.211060e+05,3583.500000
75%,0.88250,69.534250,10.360750,67.400000,0.847250,0.905250,88369.500000,4.36750,0.061000,110.000000,1.325602e+06,13398.500000
max,0.97000,76.953000,11.647000,145.900000,0.939000,0.983000,995282.000000,6.34000,0.542000,850.000000,2.971691e+07,771258.000000


In [18]:
# Check the balance of our target values
y.value_counts()

5    36
6    28
4    28
7    14
3     9
2     1
Name: ladder_score, dtype: int64

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({6: 23, 5: 26, 7: 10, 4: 21, 3: 7})

In [20]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

# Creating a StandardScaler instance.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [21]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

0.4619047619047619

In [22]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[0, 0, 1, 0, 0, 0],
       [0, 1, 1, 0, 0, 0],
       [0, 0, 4, 3, 0, 0],
       [0, 0, 3, 5, 2, 0],
       [0, 0, 0, 2, 1, 2],
       [0, 0, 0, 0, 0, 4]], dtype=int64)

In [23]:
# Print the imbalanced classification report

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          2       0.00      0.00      1.00      0.00      0.00      0.00         1
          3       1.00      0.50      1.00      0.67      0.71      0.48         2
          4       0.44      0.57      0.77      0.50      0.66      0.43         7
          5       0.50      0.50      0.74      0.50      0.61      0.36        10
          6       0.33      0.20      0.92      0.25      0.43      0.17         5
          7       0.67      1.00      0.92      0.80      0.96      0.93         4

avg / total       0.50      0.52      0.83      0.49      0.62      0.42        29



In [28]:
#export DataFrame to CSV file
df2.to_csv('happiness_top12_df.csv', index=False)